In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')

#mpl.rc('figure', figsize=(10,8))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json
import re
import os

#import copy

from Fit_data_1_22_aux import apply_filter

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *
from ur import *

from data import CAPData


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
cumsum_default=False

### Import data

In [ ]:
data_folder='./Data/SP-2021_01_22_Q395_fmaskedCAP'

fs=48828

I0 = 100 +10 - 32 #intensity ref for masker    #100 dB rms=1  +10 dB amp 5   (/sqrt(2))   #-32 dB masker atten
I0 = 10*np.log10( 10**(I0/10)/(fs/2) ) #spectral density

print(f'reference masker power spectral density (0 attn): {I0:.2f} dB')

listFiles = os.listdir(data_folder)

In [ ]:
capData=CAPData(data_folder, listFiles, begin_ind=50, end_ind=1525, mode='C+R')

In [ ]:
t=capData.t

broadband1=capData.get_signal_by_name('broadband_noise')
broadband2=capData.get_signal_by_name('broadband_noise_bis')
broadband3=capData.get_signal_by_name('broadband_noise_bis_bis')

broadband_avg=(broadband1+broadband2+broadband3)/3



nomasker1=capData.get_signal_by_name('nomasker_bis')

nomasker_avg=(nomasker1)

#pl.figure(figsize=(16,4))
pl.plot(t, broadband1)
pl.plot(t, broadband2)
pl.plot(t, broadband3)
pl.plot(t, broadband_avg, label='avg')


pl.plot(t, nomasker1, label='no masker')


#pl.xlim([0.004, 0.007])
pl.legend()



In [ ]:
#window

t0=5.7e-3
t1=9e-3
ind0=int(t0*48828)

ind0=int(t0*48828)
ind1=int(t1*48828)

win0=sg.tukey(ind1-ind0, alpha=0.4)

win=np.zeros_like(broadband_avg)
win[ind0:ind1]=win0

pl.figure()
pl.plot(t*1e3, broadband_avg, label='avg')
pl.plot(t*1e3, win*np.amax(broadband_avg))

pl.plot(t*1e3, 0.5*broadband_avg*win, label='avg')
pl.show()


In [ ]:
def process_signal(sig, cumsum=cumsum_default, return_t=False, applyfilter=True):
    
    
    if applyfilter:
        sig=apply_filter(sig)
    sig2=sig*win
    
    t0=3e-3
    t1=13e-3

    ind0=int(t0*48828)
    ind1=int(t1*48828)
    
    dim = len(np.shape(sig2))
    if dim ==1:
        sig2=sig2[ind0:ind1]
        if cumsum:
            sig2=np.cumsum(sig2)
            sig2[0:-50]*=sg.tukey(len(sig2)-50, 0.3)
            sig2[-50:]=0
    else:
        sig2=sig2[:, ind0:ind1]
        if cumsum:
            sig2=np.cumsum(sig2)
            
    
    if return_t:
        t=np.linspace(t0, t1, ind1-ind0)
        return t, sig2
    else:
        return sig2
    
    

    

In [ ]:
for plot_time in [False, True]:
    pl.figure()
    for applyfilter in [False, True]:
        t2, broadband_proc=process_signal(broadband_avg, cumsum=cumsum_default, return_t=True, applyfilter=applyfilter)
        nomasker_proc=process_signal(nomasker_avg, applyfilter=applyfilter)
        
        if plot_time:
            pl.plot(t2*1e3, nomasker_proc-broadband_proc, label= 'with filter' if applyfilter else 'without filter')
        else:
            sig2_fft= np.fft.rfft(nomasker_proc-broadband_proc)

            f=np.linspace(0, 48828/2, len(sig2_fft))
            pl.plot(f, np.abs(sig2_fft), label='with filter' if applyfilter else 'without filter')
            pl.xlim([0, 6000])

    pl.legend()

In [ ]:
def process_signal2(sig, cumsum=cumsum_default, gauss_sigma=0, applyfilter=True):
    '''subtracts the broadband noise response
    gauss_sigma: if diff of 0, smooths the signal with gaussian filter'''
    
    res = process_signal(sig-broadband_avg, cumsum=cumsum, applyfilter=applyfilter)
    if gauss_sigma !=0:
        res = gaussian_filter1d(res, gauss_sigma)
    return res
    

**Estimation ur**  
Depends on what is the focus


In [ ]:
sig=capData.get_signal_by_name('7_notch8000_bw2300_29dB')  #high freq
#sig=capData.get_signal_by_name('8_notch4000_bw1700_29dB')
#sig=capData.get_signal_by_name('9_notch3000_bw1500_29dB')
#sig=capData.get_signal_by_name('8_notch2200_bw1500_29dB') #medium freq

sig2=process_signal2(sig, applyfilter=False)

pl.plot(t2*1e3, sig2)


#ur0=sig2-broadband_proc
gauss_sigma=(0.5e-4)/(t2[1]-t2[0])
ur0=process_signal2(sig, gauss_sigma=gauss_sigma, applyfilter=True)
ur0=np.roll(ur0, -100)
pl.plot(t2*1e3, ur0)

pl.show()



In [ ]:
def deconv(released_sig, ur0=ur0, eps=1e-2):
    
    released_sig_fft=np.fft.rfft(released_sig)
    ur0_fft=np.fft.rfft(ur0)
    E_fft=released_sig_fft/(ur0_fft+eps)
    E=np.fft.irfft(E_fft)
    return E
#masked_sig=nomasker_proc-broadband_proc
masked_sig=process_signal2(nomasker_avg, gauss_sigma=gauss_sigma)
E0=deconv(masked_sig)

Estimated raw excitation pattern

In [ ]:
pl.plot(t2*1e3, E0)

Trying with projection

In [ ]:
def proj_E(E, t0=4e-3, t1=8e-3):
    '''
    constraints u between t0 and t1'''
    proj=t2>t0
    proj*=t2<t1
    return E*proj

def deconv_newton(E0, released_sig, ur0=ur0, alpha=0.02, nb_steps=20, eps_ridge=1e-1, verbose=False, t0=4e-3, t1=7e-3):
    E=proj_E(E0, t0=t0, t1=t1)

    released_sig_fft=np.fft.rfft(released_sig)
    ur0_fft=np.fft.rfft(ur0)

    E=np.expand_dims(E, axis=0)

    for i in range(nb_steps):
        E-=alpha*deconv_newton_step(E, ur0_fft, released_sig_fft, eps_ridge=eps_ridge)

        E=proj_E(E, t0=t0, t1=t1)
        E[E<0]=0
        if verbose and i%5==0:
            pl.plot(t2*1e3, E[0], label=f'step {i}')
            pl.xlabel('t (ms)')
    if verbose:
        pl.legend()
    return E[0]


E=deconv_newton(E0, masked_sig, verbose=True)


    



Narrow-band analysis

In [ ]:

s1=capData.get_signal_by_name('1_hp_10000Hz')
s2=capData.get_signal_by_name('2_hp_9000Hz')
s3=capData.get_signal_by_name('3_hp_8000Hz')
s4=capData.get_signal_by_name('4_hp_7000Hz')
s5=capData.get_signal_by_name('5_hp_6000Hz')
s6=capData.get_signal_by_name('6_hp_5000Hz')
s7=capData.get_signal_by_name('7_hp_4000Hz')
s8=capData.get_signal_by_name('8_hp_3200Hz')
s9=capData.get_signal_by_name('9_hp_2400Hz')
s10=capData.get_signal_by_name('10_hp_1800Hz')
s11=capData.get_signal_by_name('11_hp_1500Hz')
s12=capData.get_signal_by_name('12_hp_1200Hz')


s1_proc=process_signal2(s1, gauss_sigma=gauss_sigma)
s2_proc=process_signal2(s2, gauss_sigma=gauss_sigma)
s3_proc=process_signal2(s3, gauss_sigma=gauss_sigma)
s4_proc=process_signal2(s4, gauss_sigma=gauss_sigma)
s5_proc=process_signal2(s5, gauss_sigma=gauss_sigma)
s6_proc=process_signal2(s6, gauss_sigma=gauss_sigma)
s7_proc=process_signal2(s7, gauss_sigma=gauss_sigma)
s8_proc=process_signal2(s8, gauss_sigma=gauss_sigma)
s9_proc=process_signal2(s9, gauss_sigma=gauss_sigma)
s10_proc=process_signal2(s10, gauss_sigma=gauss_sigma)
s11_proc=process_signal2(s11, gauss_sigma=gauss_sigma)
s12_proc=process_signal2(s12, gauss_sigma=gauss_sigma)

In [ ]:
pl.figure()
pl.plot(s1_proc)
pl.plot(s2_proc)
pl.plot(s3_proc)
pl.plot(s4_proc)
pl.plot(s5_proc)
pl.plot(s6_proc)
pl.plot(s7_proc)
pl.plot(s8_proc)
pl.plot(s9_proc)
pl.plot(s10_proc)
pl.plot(s11_proc)
pl.plot(s12_proc)
pl.show()

pl.figure(figsize=(10,8))
pl.plot(t2*1e3,s1_proc-s2_proc, label='9-10kHz')
pl.plot(t2*1e3,s2_proc-s3_proc, label='8-9kHz')
pl.plot(t2*1e3,s3_proc-s4_proc, label='7-8kHz')
pl.plot(t2*1e3,s4_proc-s5_proc, label='6-7kHz')
pl.plot(t2*1e3,s5_proc-s6_proc, label='5-6kHz')
pl.plot(t2*1e3,s6_proc-s7_proc, label='4-5kHz')
pl.xlim([3,12])
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.show()



pl.figure(figsize=(10,8))
pl.plot(t2*1e3,s6_proc-s7_proc, label='4-5kHz')
pl.plot(t2*1e3,s7_proc-s8_proc, label='3.2-4kHz')
pl.plot(t2*1e3,s8_proc-s9_proc, label='2.4-3.2kHz')
pl.plot(t2*1e3,s9_proc-s10_proc, label='1.8-2.4kHz')
pl.plot(t2*1e3,s10_proc-s11_proc, label='1.5-1.8kHz')
pl.plot(t2*1e3,s11_proc-s12_proc, label='1.2-1.5kHz')
pl.plot(t2*1e3,s12_proc, label='-1.2kHz')
pl.xlim([3,12])
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.show()

In [ ]:
pl.figure()
i=0




pl.figure(figsize=(10,8))
for sig, label in [(s1_proc-s2_proc, '9-10kHz'),
(s2_proc-s3_proc, '8-9kHz'),
(s3_proc-s4_proc, '7-8kHz'),
(s4_proc-s5_proc, '6-7kHz'),
(s5_proc-s6_proc, '5-6kHz'),
(s6_proc-s7_proc, '4-5kHz'),(s7_proc-s8_proc, '3.2-4kHz'),
(s8_proc-s9_proc, '2.4-3.2kHz'),
(s9_proc-s10_proc, '1.8-2.4kHz'),
(s10_proc-s11_proc, '1.5-1.8kHz'),
(s11_proc-s12_proc, '1.2-1.5kHz'),
(s12_proc, '-1.2kHz')]:
    E=deconv(sig, eps=1e-2)
    E=deconv_newton(E, sig, alpha=0.005, nb_steps=50, eps_ridge=2e-1, t0=4.3e-3, t1=7e-3)
    pl.plot(t2*1e3, E-0.25*i, label=label)
    i+=1

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.xlim([4, 8])
pl.show()

In [ ]:
#NB: some data not clear
t_max=np.array([43.5,43.5,45,42,45.5,46.5,48,51,56,65,74,82])
#t_max_bis=np.array([23,26,25.5,25.5,28.5,29])



t_max_C=np.array([44,51.5,47,46,52,54,49,52,65,73,77,96.5])   #mod 0

t_max_R=np.array([44,49.5,47.5,45,49.5,51,53.5,61,76.5,88,104,122])


#t_0lat=4e-3-3.6e-3+2e-3
t_0lat=4e-3+2e-3-6e-3


t_max=t_0lat+t_max*2*1e-5
t_max_C=t_0lat+t_max_C*2*1e-5

t_max_R=t_0lat+t_max_R*2*1e-5

#t_max_bis=t_0lat+t_max_bis*2*1e-5
freqs=np.array([9.5,8.5,7.5,6.5,5.5,4.5,3.6,2.8,2.1,1.65, 1.35, 1])
pl.plot(freqs, t_max*1e3, '+', markersize=12, label='C+R')
pl.plot(freqs, t_max_C*1e3, '+', markersize=12, label='C')

pl.plot(freqs, t_max_R*1e3, '+', markersize=12, label='R')


#pl.plot(freqs[0:len(t_max_bis)], t_max_bis*1e3, '+', markersize=12, label='C+R (first peak?)')

pl.ylabel('Estimated latencies (ms)')
pl.xlabel('freq (kHz)')

pl.legend()
#NB CM begins at 6 ms approx
#peak convol begins at 5-3 ms = 2ms (C) approx

fit latencies 2 power laws

In [ ]:
#above 4kHz


freqs_pts=np.array([9.5,8.5,7.5,6.5,5.5,4.5])*1e3

inds=np.array([0,1,2, 4,5]) #HACK remove outlier 6.5 kHz
freqs_pts=freqs_pts[inds]
freqs_pts0=freqs_pts
t_max_pts=t_max[inds]
t_max_pts0=t_max_pts


lat=PowerLawLatencies(1e6, alpha=1, t0=4e-3, mode='left')
lat.fit_data(t_max_pts, freqs_pts, init_with_new_values=False, bounds=[0.5, 2])

freqs_lin=np.linspace(4, 10)*1e3
pl.plot(freqs_lin, lat(freqs_lin)*1e3)

pl.plot(freqs_pts, t_max_pts*1e3, '+', markeredgewidth=3, markersize=10)

lat_above4k=lat


#below 4kHz
freqs_pts=freqs[6:]*1e3
t_max_pts=t_max[6:]
lat=PowerLawLatencies()
lat.fit_data(t_max_pts, freqs_pts)

freqs_lin=np.linspace(0.5, 5)*1e3
pl.plot(freqs_lin, lat(freqs_lin)*1e3, color='C2')


freqs_lin=np.linspace(0.5, 10)*1e3
pl.plot(freqs_lin, lat(freqs_lin)*1e3, color='C2', linestyle='--')

pl.plot(freqs_pts, t_max_pts*1e3, '+', markeredgewidth=3, markersize=10)

pl.show()

pl.figure()

pl.plot(freqs_lin, (lat(freqs_lin)-lat.t0)*1e3, color='C2', linestyle='--')

pl.plot(freqs_pts0, (t_max_pts0-lat.t0.numpy())*1e3, '+', markeredgewidth=3, markersize=10)

pl.plot(freqs_pts, (t_max_pts-lat.t0.numpy())*1e3, '+', markeredgewidth=3, markersize=10)

pl.ylabel(' t - t_0 (ms)')

pl.xlabel(' f (Hz)')

pl.xscale('log')
pl.yscale('log')




In [ ]:
lat

### 4 kHz

first estimation I/O curve

In [ ]:

cap=[]
rms=[]

masker_list=['2_notch4000_bw1700_55dB',
'3_notch4000_bw1700_50dB',
'4_notch4000_bw1700_45dB',
'5_notch4000_bw1700_40dB',
'6_notch4000_bw1700_35dB',
'7_notch4000_bw1700_32dB',
'8_notch4000_bw1700_29dB',
'9_notch4000_bw1700_26dB',
'10_notch4000_bw1700_23dB']

for masker in masker_list:
    sig=capData.get_signal_by_name(masker)
    sig=process_signal(sig)
    broadband_sig_trunc=process_signal(broadband2)
    
    #REF broadband
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    #rms.append(np.std(sig-broadband_sig_trunc))
    
    #cap.append(np.max(sig)-np.min(sig))
    #rms.append(np.std(sig-broadband_sig_trunc))
    pl.plot(t2, sig-broadband_sig_trunc, label=masker)


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.show()

pl.figure(figsize=(8, 6))

attns=-np.array([55, 50, 45,40,35,32,29,26,23])   # 20])
pl.plot(attns, cap, '+', label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('Notch attenuation')
pl.ylabel('Amplitude difference')


In [ ]:
sigm=SigmoidIOFunc(0, 0)
#maskamount=1-(  (cap-np.amin(cap)) /np.amax(cap-np.amin(cap)) )
maskamount=1-(cap/np.amax(cap)) 

I_pts=I0+attns
#sigm.fit_data(I_pts, maskamount, set_mmax=True)

#HACK enforce masking=100% at attn20
#sigm.mmax.data*=1/sigm(I0-20)

sigm.fit_data(I_pts, maskamount, constrained_at_Iref=True, Iref=I0-20)

I=np.linspace(-30, 20)
pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)

pl.plot(I0-20, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (max: broadband)')

Setting model

In [ ]:
ntch_maskerNames, ntch_maskingConds, ntch_signals =capData.get_batch_re('.*notch4000_bw1700')
ntch_maskingConds.set_amp0_dB(I0)

In [ ]:
#HACK pad maskers >12e3 to avoid issues with latencies (equivalent to taking the difference
#  excitations of maskers - excitation 'broadband noise')
ntch_maskingConds.pad_maskers(f_thr=11000, f_max=1e5)

In [ ]:
gauss_sigma=(1.5e-4)/(t2[1]-t2[0])
ntch_signals_proc=process_signal2(ntch_signals, gauss_sigma=gauss_sigma)

In [ ]:
# test

for maskerName, sig in zip(ntch_maskerNames, ntch_signals_proc):
    pl.plot(t2, sig, label=maskerName)

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# estimation ur

sig=capData.get_signal_by_name('4_notch8000_bw2300_45dB')
sig2=process_signal(sig)

pl.plot((t2-3e-3)*1e3, sig2-broadband_proc)

#pl.plot(t2*1e3, sig2bis-broadband_proc)

t_shift=6e-3-3e-3 #excitation coincides with CM

ur0=sig2-broadband_proc
ur0=np.roll(ur0,  -int(t_shift*48828) )
pl.plot((t2-3e-3)*1e3, ur0)
#pl.xlim([4,6])

In [ ]:
m=72
E0_temp=sg.windows.tukey(m, alpha=0.5) 
E0=np.zeros_like(t2)
ind_begin=int((t_shift-1e-3)*48828)
ind_end=int((t_shift-1e-3)*48828)+m
E0[ind_begin:ind_end]=E0_temp

pl.plot(t2*1e3, E0)
pl.title('Init raw excitation')
pl.xlabel('t (ms)')
pl.ylabel('Amp')

In [ ]:
#latencies model

#lat_model=lat_above4k
#lat_model=lat

#lat_model=PowerLawLatencies.shift(lat_model, 6e-3-1e-3)   #t0: start CM-1ms

#HACK as latencies are very small (sampling issues), manual dilatation

lat_model=PowerLawLatencies.fromPts(0.0058, 6000, 0.006, 4000)

In [ ]:
E=ExcitationPatterns(t2, E0)  #no non-maskable part

BW10_0=1000
BW10_0Func=constant_BW10(BW10_0, requires_grad=False)

E.set_masking_model(lat_model, BW10_0Func, ntch_maskingConds, sigm)

In [ ]:
plotLatencies(lat_model)

In [ ]:
pl.figure(figsize=(10,20))
plotExcitationPatterns(E, plot_raw_excitation=True) # ylim_top=1
pl.show()

In [ ]:
#estimation ur

maskAmounts, excs = E.get_tensors() 

nb_steps=20
alpha=np.linspace(0.5, 0.05, nb_steps)

EPs_fft=np.fft.rfft(excs, axis=1)
CAPs_fft=np.fft.rfft(ntch_signals_proc, axis=1)
#u1_mat=np.tile(ur0, (ntch_maskingConds.n_conditions, 1))
u1_mat=np.zeros_like(ntch_signals_proc)
filter_mat  = (t2>7.5e-3)+(t2<3.2e-3)
filter_mat=np.tile(filter_mat, (ntch_maskingConds.n_conditions, 1))
#filter_mat=np.zeros_like(ntch_signals_proc, dtype=bool)
#proj_fft=E.get_projector_fft()

weights=np.sqrt(np.sum(excs.clone().detach().numpy()**2, axis=1))
for i in range(1, nb_steps+1):
    du=deconv_newton_step(u1_mat, EPs_fft, CAPs_fft, eps_ridge=0)   #TODO proj_fft
    #du=deconv_grad(u1_mat, EPs_fft, CAPs_fft)
    
    u1_mat-=alpha[i-1]*du
    #proj 1 
    u1_mat[filter_mat]=np.zeros_like(u1_mat[filter_mat])
    #proj 2

    #u1_mat_mean=np.mean(u1_mat, axis=0)[None, :]
    
    
    #HACK waiting for proj_fft
    u1_mat_mean=np.average(u1_mat, axis=0, weights=weights)[None, :]
    u1_mat=np.repeat(u1_mat_mean, ntch_maskingConds.n_conditions, axis=0)
     
    '''
    for i in range(5):
            pl.figure()
            name=ntch_maskerNames[i]
            pl.plot(u1_mat[i], label=name, color=f'C{i}')
            #pl.plot( np.abs(EPs_fft[i]))
            pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    '''
    
    if i==nb_steps:
        pl.figure()
        pl.title(f'Step {i} (deconv + proj)')
        #pl.plot(t, u0, label='u0 (truth)')
        pl.plot(t2, u1_mat[0], label='u0 (estimated)')
        pl.legend()
        pl.show()

Reestimation I/O curve

In [ ]:
exc_max=[]
exc_rms=[]
attns_=[]

amps=ntch_maskingConds.amp_list[1]
for i in range(len(ntch_maskerNames)):
    name=ntch_maskerNames[i]
    sig=ntch_signals_proc[i]
    
    #pl.plot(t2*1e3, excs[i], label=name, color=f'C{i}')

    E_i=np.zeros_like(sig)
    E_i=deconv_newton(E_i, sig, ur0=u1_mat[0], alpha=1, nb_steps=20, t0=5.7e-3, t1=6.5e-3, eps_ridge=0.1)  #double peak?
    
    
    attn_=20*np.log10(amps[i])
    
    #HACK remove outlier
    #if not (np.abs(attn_-65)<2):
    if True:
        exc_max.append(np.amax(E_i))

        exc_rms.append(np.sqrt(np.sum(E_i**2) ))
        attns_.append(attn_)
    pl.plot(t2*1e3, E_i, label=name, color=f'C{i}')
    
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


pl.figure()
pl.plot(attns_, exc_max, '+')
                   
pl.plot(attns_, np.array(exc_rms)*0.2, '+')
pl.xlabel('Intensity (dB)')

In [ ]:
#reestimation I-0 curve: 

I_pts=np.array(attns_)  #I0 already in masking cond

sigm=SigmoidIOFunc(0, 0)
exc_rms=np.array(exc_rms)
maskamount=1- (exc_rms)/np.amax(exc_rms) 


#add broadband condition
#attns_.append(80)
#ma_list=list(maskamount)
#ma_list.append(1.)
#maskamount=np.array(ma_list)

#sigm.fit_data(np.array(attns_), maskamount, set_mmax=True)

#HACK enforce masking=100% at attn20
#sigm.mmax.data*=1/sigm(I0-20)

sigm.fit_data(I_pts, maskamount, constrained_at_Iref=True, Iref=I0-20)


I=np.linspace(-30, 20)
pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')


pl.suptitle('Amount of masking in response to broadband noise')
pl.title(' (as estimated with the notch method)', fontsize=10)
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)
pl.plot(I0-20, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (ref max: no notch)')


test model

In [ ]:
#model
u1=u1_mat[0]
pl.figure(figsize=(12,20))
ax_list=plotSimulatedCAPs(E, u1, ylim=[-0.01, 0.01], max_plots=10)
plotSimulatedCAPs(E, CAParray=ntch_signals_proc, axlist=ax_list, max_plots=10)
pl.plot()

In [ ]:
fln_list=['1-notch3800_bw800.json',
'2-notch4000_bw900.json',
'3-notch4000_bw1000.json',
'4-notch4200_bw1100.json',
'5-notch4200_bw800.json',
'6-notch3800_bw1200.json']
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
reg_exp=')|('.join(masker_list)
reg_exp='('+reg_exp+')'
vbw_maskerNames, vbw_maskingConds, vbw_signals =capData.get_batch_re(reg_exp)
vbw_signals_proc=process_signal2(vbw_signals, gauss_sigma=gauss_sigma)
vbw_maskingConds.set_amp0_dB(I0)

In [ ]:
#HACK
vbw_maskingConds.pad_maskers(f_thr=11000, f_max=1e5)

In [ ]:
E2=ExcitationPatterns(t2, E0)  #no non-maskable part
BW10_0TestFunc=constant_BW10(1200, requires_grad=False)

E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, sigm)

In [ ]:
#model
u1=u1_mat[0]
pl.figure(figsize=(10,14))
ax_list=plotSimulatedCAPs(E2, u1, ylim=[-0.01, 0.01])
plotSimulatedCAPs(E2, CAParray=vbw_signals_proc, axlist=ax_list)
pl.plot()

In [ ]:
bw_arr=np.linspace(500, 4000)
sigs_ref=vbw_signals_proc
errs=[]
for bw in bw_arr:

    BW10_0TestFunc=constant_BW10(bw, requires_grad=False)
    E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, sigm)
    excs = E2.get_tensor() 
    maskingConditions = E2.maskingConditions
    err=0
    for i, exc in zip(range(maskingConditions.n_conditions), excs):
        exc_np = exc.detach().numpy()
        CAP=np.convolve(exc_np, u1, mode='full')
        t=E.t.numpy()
        CAP=CAP[0:len(E2.t)]
        err+=np.sum( (CAP-sigs_ref[i])**2)
    errs.append(err)
    
pl.plot(bw_arr, errs)
pl.xlabel('BW10 model (Hz)')

pl.ylabel('square err')
